# Leiteroperatoren

## Vorbereitung

Führen sie zuerst die folgende Zelle aus, um die Simulation vorzubereiten. Sie können diese danach mit dem Pfeil links einklappen. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class leiter_operatoren():
    def __init__(self, hbar=1, m=1, omega=1, x_min=-10, x_max=10, Nx=2**9):
        self.hbar = hbar
        self.m = m
        self.omega=omega
    
        self.x_min = x_min
        self.x_max = x_max
        self.Nx = Nx # Less grid points increases stability after several a/a^dagger operations
    
        self.dx = (x_max-x_min)/(Nx)#Nx-1???
        self.x_grid = np.arange(self.x_min,self.x_max,self.dx)
    
    def init_psi(self):
        self.psi = np.zeros(self.Nx, dtype=complex)
        self.psi[:] = self.m * self.omega / (np.sqrt(2*np.pi) * self.hbar) * np.exp( - self.m * self.omega * self.x_grid**2 / (2 * self.hbar) )
        self.n = 0
    
    def diff_psi(self, psi):
        d_psi = np.zeros(self.Nx,dtype=complex)
        d_psi[0] = (psi[1] - psi[0]) / self.dx
        d_psi[-1] = (psi[-1] - psi[-2]) / self.dx
        for i in range(1,self.Nx-1):
            d_psi[i] = (psi[i+1] - psi[i-1]) / (2 * self.dx)
        return d_psi
    
    def apply_a(self):
        d_psi = self.diff_psi(self.psi)
        self.psi = (np.sqrt(self.m * self.omega / self.hbar) * self.x_grid * self.psi + np.sqrt(self.hbar / (self.m * self.omega)) * d_psi) / np.sqrt(2)
        self.norm_a()
        self.n += -1
        
    
    def apply_a_dagger(self):
        d_psi = self.diff_psi(self.psi)
        self.psi = ( np.sqrt(self.m * self.omega / self.hbar) * self.x_grid * self.psi - np.sqrt(self.hbar / (self.m * self.omega)) * d_psi) / np.sqrt(2)
        self.norm_a_dagger()
        self.n += 1
    
    def norm_a(self):
        if self.n > 0:
            self.psi = self.psi / np.sqrt(self.n)
        #else: return self.psi
        
    def norm_a_dagger(self):
        if self.n > 0:
            self.psi= self.psi / np.sqrt(self.n+1)
        #else: return psi
    
    def calc_rho(self,psi):
        return np.abs(psi)**2
    
    def plot_psi(self):
        rho = self.calc_rho(self.psi)
        
        fig = make_subplots(rows=2, cols=1,
                            subplot_titles=['Wellenfunktion im Ortsraum', 'Aufenthaltswahrscheinlichkeit im Ortsraum'],
                            )
        
        fig.add_trace(go.Scatter(x=self.x_grid, y=np.real(self.psi[:]), 
                                 name=r'$\text{Re}\left(\psi\left(x,t\right)\right)$'), 1, 1)
        fig.add_trace(go.Scatter(x=self.x_grid, y=np.imag(self.psi[:]),
                                 name=r'$\text{Im}\left(\psi\left(x,t\right)\right)$'), 1, 1)
        fig.add_trace(go.Scatter(x=self.x_grid, y=rho[:],
                                 name=r'$|\psi(x,t)|^2$'), 2, 1)
        
        fig.update_xaxes(range=[self.x_min, self.x_max])
        fig.update_yaxes(range=[-0.5, 0.5])
        fig.update_yaxes(range=[0,0.5],row=2,col=1)
        fig.update_layout(width=1000, height=700)
        fig.show()
    
    
    def display(self):
        a_button = widgets.Button(description='Absteiger')
        a_dagger_button = widgets.Button(description='Aufsteiger')
        reset_button = widgets.Button(description='Reset')

        def a_button_function(click_input):
            with out:
                clear_output()
                self.apply_a()
                self.plot_psi()
                        
        a_button.on_click(a_button_function)

        def a_dagger_button_function(click_input):
            with out:
                clear_output()
                self.apply_a_dagger()
                self.plot_psi()
                
        a_dagger_button.on_click(a_dagger_button_function)
                
        def reset_button_function(click_input):
            with out:
                clear_output()
                self.init_psi()
                self.plot_psi()
                
        reset_button.on_click(reset_button_function)

        out = widgets.Output()
        reset_button_function('')

        #### Not visible figure because Colab cannot deal with these buttons plus plotly figures at once ####
        #### But with this its tricked into being displayed properly ########################################
        fig_placeholder = go.Figure()
        fig_placeholder.add_trace(go.Scatter(x=[1], y=[1], 
                                     ))
        fig_placeholder.update_layout(width=10, height=10)
        fig_placeholder.show()
        ######################################################################################################

        display(widgets.HBox([a_button,a_dagger_button,reset_button] ),out)

        reset_button_function(1)

## Simulation

Durch das ausführen der nächsten Zeile starten sie die Simulation.

In [ ]:
leiter = leiter_operatoren()
leiter.display()

## Schritt für Schritt Berechnung

In [ ]:
hbar = 1
m = 1
omega = 1

### Initialisierung des diskreten 1D Koordinatensystems

In [ ]:
x_min = -10
x_max = 10
Nx = 2**10
dx = (x_max-x_min)/(Nx)
x_grid = np.arange(x_min,x_max,dx)

### Wellenfunktion initialisieren

Die Wellenfunktion des Grundzustandes des harmonischen Oszillators, gegeben als

$\psi_0 (x) = \frac{m\omega}{\sqrt{2\pi\hbar^2} } e^{-\frac{1}{2} \frac{m\omega}{\hbar} x^2},$

wird initialisiert.

In [ ]:
def init_psi():
    psi = np.zeros(Nx,dtype=complex)
    psi[:] = m * omega / (np.sqrt(2*np.pi) * hbar) * np.exp( - m * omega * x_grid**2 / (2 * hbar) ) 
    return psi

### Ableitung als Differenzenquotient

Die Ableitung von $\Psi(x)$ wird an jedem Gitterpunkt mithilfe des zentralen Differenzenquotienten berechnet:

$\frac{d\Psi(x)}{dx}\big|_{x=x_i} = \frac{\Psi(x_{i+1}) - \Psi(x_{i-1})}{2\Delta x}$ 

Mit Ausnahme der Randpunkte, wo am linken Rand der Vorwärts-Differenzenquotient und am rechten Rand der Rückwärts-Differenzenquotient verwendet werden.

In [ ]:
def diff_psi(psi):
    d_psi = np.zeros(Nx,dtype=complex)
    d_psi[0] = (psi[1] - psi[0]) / dx
    d_psi[-1] = (psi[-1] - psi[-2]) / dx
    for i in range(1,Nx-1):
        d_psi[i] = (psi[i+1] - psi[i-1]) / (2 * dx)
    return d_psi

### Leiteroperatoren

Definition der Leiteroperatoren für den Harmonischen Oszillator

$\hat{a}^{\dagger} \psi(x) = \frac{1}{\sqrt{2}} \left( \sqrt{\frac{m\omega}{\hbar}} x - \sqrt{\frac{\hbar}{m\omega}} \frac{d}{dx}\right) \psi(x)$

$\hat{a} \psi(x) = \frac{1}{\sqrt{2}} \left( \sqrt{\frac{m\omega}{\hbar}} x + \sqrt{\frac{\hbar}{m\omega}} \frac{d}{dx}\right) \psi(x)$

In [ ]:
def apply_a(psi):
    d_psi = diff_psi(psi)
    return (np.sqrt(m*omega/hbar) * x_grid * psi + np.sqrt(hbar/(m*omega)) * d_psi) / np.sqrt(2)

In [ ]:
def apply_a_dagger(psi):
    d_psi = diff_psi(psi)
    return ( np.sqrt(m*omega/hbar) * x_grid * psi - np.sqrt(hbar/(m*omega)) * d_psi) / np.sqrt(2)

### Normierung des Eigenzustandes

Aus der allgemeinen Wirkung von Leiteroperatoren auf Eigenzustände:

$\hat{a} |n\rangle = \sqrt{n} |n-1\rangle$

$\hat{a}^{\dagger} |n\rangle = \sqrt{n+1} |n+1\rangle$

folgt, dass die Wellenfunktion nach Wirkung der Leiteroperatoren folgendermaßen normiert werden müssen, um wieder einen normierten Eigenzustand darzustellen. (???)

In [ ]:
def norm_a(psi, n):
    if n > 0:
        return psi / np.sqrt(n)
    else: return psi

In [ ]:
def norm_a_dagger(psi, n):
    if n > 0:
        return psi / np.sqrt(n+1)
    else: return psi

### Wahrscheinlichkeitsdichte

Berechnung der Wahrscheinlichkeitsdichte im Ortsraum:

$\rho(x) = |\Psi(x)|^2$ 

In [ ]:
def calc_rho(psi):
     return np.abs(psi)**2

### Plotten

In [ ]:
def plot_psi(psi):
    rho = calc_rho(psi)
    
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=['Wellenfunktion im Ortsraum', 'Aufenthaltswahrscheinlichkeit im Ortsraum'],
                        )
    
    fig.add_trace(go.Scatter(x=x_grid, y=np.real(psi[:]), 
                             name=r'$\text{Re}\left(\psi\left(x,t\right)\right)$'), 1, 1)
    fig.add_trace(go.Scatter(x=x_grid, y=np.imag(psi[:]),
                             name=r'$\text{Im}\left(\psi\left(x,t\right)\right)$'), 1, 1)
    fig.add_trace(go.Scatter(x=x_grid, y=rho[:],
                             name=r'$|\psi(x,t)|^2$'), 2, 1)
    
    fig.update_xaxes(range=[x_min, x_max])
    fig.update_yaxes(range=[-0.5, 0.5])
    fig.update_yaxes(range=[0,0.5],row=2,col=1)
    fig.update_layout(width=1000, height=700)
    fig.show()

In [ ]:
class psi_tracker_class:
    def __init__(self):
        self.psi = init_psi()
        self.n = 0
    
    def a_click(self):
        self.psi = apply_a(self.psi)
        self.psi = norm_a(self.psi, self.n)
        self.n += -1
    
    def a_dagger_click(self):
        self.psi = apply_a_dagger(self.psi)
        self.psi = norm_a_dagger(self.psi, self.n)
        self.n += 1
            
    def reset_click(self):
        self.psi = init_psi()
        self.n = 0
        
    def plot(self):
        plot_psi(self.psi)

In [ ]:
psi_tracker = psi_tracker_class()

a_button = widgets.Button(description='Absteiger')
a_dagger_button = widgets.Button(description='Aufsteiger')
reset_button = widgets.Button(description='Reset')

def a_button_function(click_input):
    with out:
        clear_output()
        psi_tracker.a_click()
        psi_tracker.plot()
                
a_button.on_click(a_button_function)

def a_dagger_button_function(click_input):
    with out:
        clear_output()
        psi_tracker.a_dagger_click()
        psi_tracker.plot()
        
a_dagger_button.on_click(a_dagger_button_function)
        
def reset_button_function(click_input):
    with out:
        clear_output()
        psi_tracker.reset_click()
        psi_tracker.plot()
        
reset_button.on_click(reset_button_function)

out = widgets.Output()
reset_button_function('')

#### Not visible figure because Colab cannot deal with these buttons plus plotly figures at once ####
#### But with this its tricked into being displayed properly ########################################
fig_placeholder = go.Figure()
fig_placeholder.add_trace(go.Scatter(x=[1], y=[1], 
                             ))
fig_placeholder.update_layout(width=10, height=10)
fig_placeholder.show()
######################################################################################################

display(widgets.HBox([a_button,a_dagger_button,reset_button] ),out)

reset_button_function(1)